# Import dependencies

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time

# Download data

## Set transforms

In [14]:
transform = transforms.Compose([
    transforms.Resize((228, 228)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [15]:
training_data = CIFAR10(
    root='data',
    train=True,
    transform=transform,
    download=True
)

test_data = CIFAR10(
    root='data',
    train=True,
    transform=transform,
    download=True
)

Files already downloaded and verified
Files already downloaded and verified


# Define AlexNet network

In [3]:
class AlexNet(nn.Module):
    def __init__(self, in_channels=3, num_cls=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=96,
                kernel_size=(11, 11),
                stride=4,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=(3, 3),
                stride=2
            )
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(
                in_channels=96,
                out_channels=256,
                kernel_size=(5, 5),
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=(3, 3),
                stride=2
            )
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(
                in_channels=256,
                out_channels=384,
                kernel_size=(3, 3),
                padding=1
            ),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(
                in_channels=384,
                out_channels=384,
                kernel_size=(3, 3),
                padding=1,
                stride=1
            ),
            nn.ReLU()
        )
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(
                in_channels=384,
                out_channels=256,
                kernel_size=(3, 3),
                padding=1,
                stride=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=(3, 3),
                stride=2
            )
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(4096, num_cls)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# Training

## Hyperparameters

In [16]:
LR = 0.001
EPOCHS = 3
BATCH_SIZE = 64

TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.25

TRAIN_SIZE = int(TRAIN_SPLIT * len(training_data))
VAL_SIZE = int(VAL_SPLIT * len(training_data))

training_data, val_data = random_split(
    training_data,
    [TRAIN_SIZE, VAL_SIZE],
    torch.Generator().manual_seed(42)
)

## Dataloader

In [17]:
train_dataloader = DataLoader(
    dataset=training_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
)
val_dataloader = DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
)

## For plotting

In [18]:
matplotlib.use("Agg")
H = {
    'avg_train_loss': [],
    'train_acc': [],
    'avg_val_loss': [],
    'val_acc': [],
}
train_steps = len(train_dataloader) / BATCH_SIZE
val_steps = len(val_dataloader) / BATCH_SIZE

## Model, loss function and optimizer
I use Cross Entropy Loss with Adam optimizer

In [19]:
model = AlexNet()
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR
)

## Train loop

In [ ]:
print('[INFO] Start training the network ...')
start_time = time.time()
for t in range(EPOCHS):
    total_train_loss = 0
    total_val_loss = 0
    train_correct = 0
    val_correct = 0

    # training
    model.train()
    for (inputs, labels) in train_dataloader:
        # forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss
        train_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    avg_train_loss = total_train_loss / train_steps
    train_correct /= len(train_dataloader.dataset)

    # validation
    with torch.no_grad():
        for (inputs, labels) in val_dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss
            val_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    avg_val_loss = total_val_loss / val_steps
    val_correct /= len(val_dataloader.dataset)

    # history
    H['avg_train_loss'].append(avg_train_loss)
    H['avg_val_loss'].append(avg_val_loss)
    H['train_acc'].append(train_correct)
    H['val_acc'].append(val_correct)

    print(f'[INFO]: Epoch {t + 1}/{EPOCHS}')
    print(f'Train loss: {avg_train_loss:.6f}, train accuracy: {train_correct:4f}%')
    print(f'Val loss: {avg_val_loss:.6f}, val accuracy: {val_correct:4f}%')

end_time = time.time()
print(f'[INFO] Training taken {end_time - start_time:.4f}')

[INFO] Start training the network ...
[INFO]: Epoch 1/3
Train loss: 109.419472, train accuracy: 0.365840%
Val loss: 91.500542, val accuracy: 0.476480%
